In [1]:
import tensorflow as tf
from tensorflow.contrib.rnn import GRUCell

/home/eurismar/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/eurismar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
flags = tf.app.flags

In [3]:
flags.DEFINE_integer('GEN_STATE_SIZE', 512, '')

In [4]:
flags.DEFINE_float('NOISE_STDEV', 10.0, '')

In [5]:
flags.DEFINE_integer('GEN_GRU_LAYERS', 1, '')

In [6]:
FLAGS = flags.FLAGS

In [7]:
BATCH_SIZE = 64

In [8]:
def make_noise(shape, mean=0.0, stddev=1.0):
    return tf.random_normal(shape, mean, stddev)

In [9]:
def get_noise():
    noise_shape = [BATCH_SIZE, FLAGS.GEN_STATE_SIZE]
    return make_noise(shape=noise_shape, stddev=FLAGS.NOISE_STDEV), noise_shape

In [10]:
def create_initial_states(noise):
    states = []
    for l in range(FLAGS.GEN_GRU_LAYERS):
        states.append(noise)
    return states

In [11]:
def get_train_op(cells, char_input, charmap_len, embedding, gt, n_samples, num_neurons, seq_len, sm_bias, sm_weight,
                 states):
    gt_embedding = tf.reshape(gt, [n_samples * seq_len, charmap_len])
    gt_GRU_input = tf.matmul(gt_embedding, embedding)
    gt_GRU_input = tf.reshape(gt_GRU_input, [n_samples, seq_len, num_neurons])[:, :-1]
    gt_sentence_input = tf.concat([char_input, gt_GRU_input], axis=1)
    GRU_output, _ = rnn_step_prediction(cells, charmap_len, gt_sentence_input, num_neurons, seq_len, sm_bias,
                                         sm_weight,
                                         states)
    train_pred = []
    # TODO: optimize loop
    for i in range(seq_len):
        train_pred.append(
            tf.concat([tf.zeros([BATCH_SIZE, seq_len - i - 1, charmap_len]), gt[:, :i], GRU_output[:, i:i + 1, :]],
                      axis=1))

    train_pred = tf.reshape(train_pred, [BATCH_SIZE*seq_len, seq_len, charmap_len])

    if FLAGS.LIMIT_BATCH:
        indices = tf.random_uniform([BATCH_SIZE], 0, BATCH_SIZE*seq_len, dtype=tf.int32)
        train_pred = tf.gather(train_pred, indices)

    return train_pred


In [87]:
def rnn_step_prediction(cells, charmap_len, gt_sentence_input, num_neurons, seq_len, sm_bias, sm_weight, states,
                        reuse=False):
    with tf.variable_scope("rnn", reuse=reuse):
        GRU_output = gt_sentence_input
        for l in range(FLAGS.GEN_GRU_LAYERS):
            GRU_output, states[l] = tf.nn.dynamic_rnn(cells[l], GRU_output, dtype=tf.float32,
                                                       initial_state=states[l], scope="layer_%d" % (l + 1))
    GRU_output = tf.reshape(GRU_output, [-1, num_neurons])
    GRU_output = tf.nn.softmax(tf.matmul(GRU_output, sm_weight) + sm_bias)
    GRU_output = tf.reshape(GRU_output, [BATCH_SIZE, -1, charmap_len],name='GRU_output')
    return GRU_output, states

In [88]:
def get_inference_op(cells, char_input, embedding, seq_len, sm_bias, sm_weight, states, num_neurons, charmap_len,
                     reuse=False):
    inference_pred = []
    embedded_pred = [char_input]
    for i in range(seq_len):
        step_pred, states = rnn_step_prediction(cells, charmap_len, tf.concat(embedded_pred, 1), num_neurons, seq_len,
                                                sm_bias, sm_weight, states, reuse=reuse)
        best_chars_tensor = tf.argmax(step_pred, axis=2)
        best_chars_one_hot_tensor = tf.one_hot(best_chars_tensor, charmap_len)
        best_char = best_chars_one_hot_tensor[:, -1, :]
        inference_pred.append(tf.expand_dims(best_char, 1))
        embedded_pred.append(tf.expand_dims(tf.matmul(best_char, embedding), 1))
        reuse = True  # no matter what the reuse was, after the first step we have to reuse the defined vars

    return tf.concat(inference_pred, axis=1)

In [89]:
def get_train_op(cells, char_input, charmap_len, embedding, gt, n_samples, num_neurons, seq_len, sm_bias, sm_weight,
                 states):
    gt_embedding = tf.reshape(gt, [n_samples * seq_len, charmap_len])
    gt_GRU_input = tf.matmul(gt_embedding, embedding)
    gt_GRU_input = tf.reshape(gt_GRU_input, [n_samples, seq_len, num_neurons])[:, :-1]
    gt_sentence_input = tf.concat([char_input, gt_GRU_input], axis=1)
    GRU_output, _ = rnn_step_prediction(cells, charmap_len, gt_sentence_input, num_neurons, seq_len, sm_bias,
                                         sm_weight,
                                         states)
    train_pred = []
    # TODO: optimize loop
    for i in range(seq_len):
        train_pred.append(
            tf.concat([tf.zeros([BATCH_SIZE, seq_len - i - 1, charmap_len]), gt[:, :i], GRU_output[:, i:i + 1, :]],
                      axis=1))

    train_pred = tf.reshape(train_pred, [BATCH_SIZE*seq_len, seq_len, charmap_len])

    if FLAGS.LIMIT_BATCH:
        indices = tf.random_uniform([BATCH_SIZE], 0, BATCH_SIZE*seq_len, dtype=tf.int32)
        train_pred = tf.gather(train_pred, indices)

    return train_pred

In [90]:
def Generator_GRU_CL_VL_TH(n_samples, charmap_len, seq_len=None, gt=None):
    with tf.variable_scope("Generator"):
        noise, noise_shape = get_noise()
        num_neurons = FLAGS.GEN_STATE_SIZE
        cells = []
        for l in range(FLAGS.GEN_GRU_LAYERS):
            cells.append(GRUCell(num_neurons))
        # this is separate to decouple train and test
        train_initial_states = create_initial_states(noise)
        inference_initial_states = create_initial_states(noise)
        
        sm_weight = tf.Variable(tf.random_uniform([num_neurons, charmap_len], minval=-0.1, maxval=0.1))
        sm_bias = tf.Variable(tf.random_uniform([charmap_len], minval=-0.1, maxval=0.1))
        
        embedding = tf.Variable(tf.random_uniform([charmap_len, num_neurons], minval=-0.1, maxval=0.1))

        char_input = tf.Variable(tf.random_uniform([num_neurons], minval=-0.1, maxval=0.1))
        char_input = tf.reshape(tf.tile(char_input, [n_samples]), [n_samples, 1, num_neurons])
        if seq_len is None:
            seq_len = tf.placeholder(tf.int32, None, name="ground_truth_sequence_length")
            
        if gt is not None: #if no GT, we are training
            train_pred = get_train_op(cells, char_input, charmap_len, embedding, gt, n_samples, num_neurons, seq_len,
                                      sm_bias, sm_weight, train_initial_states)
            inference_op = get_inference_op(cells, char_input, embedding, seq_len, sm_bias, sm_weight, inference_initial_states,
                                            num_neurons,
                                            charmap_len, reuse=True)
        else:
            inference_op = get_inference_op(cells, char_input, embedding, seq_len, sm_bias, sm_weight, inference_initial_states,
                                            num_neurons,
                                            charmap_len, reuse=False)
            train_pred = None    
            
        
        return train_pred, inference_op

In [217]:
def Generator_GRU_CL_VL_TH2(n_samples, charmap_len, seq_len=None, gt=None):
    with tf.variable_scope("Generator2"):
        noise_shape = [BATCH_SIZE, FLAGS.GEN_STATE_SIZE]
        noise = tf.random_normal(shape=noise_shape, mean=0.0, stddev=1.0,name='noise')
        num_neurons = FLAGS.GEN_STATE_SIZE
        cells = []
        for l in range(FLAGS.GEN_GRU_LAYERS):
            cells.append(GRUCell(num_neurons))
        
        states = []
        for l in range(FLAGS.GEN_GRU_LAYERS):
            states.append(noise)
        train_initial_states = states
        
        states = []
        for l in range(FLAGS.GEN_GRU_LAYERS):
            states.append(noise)
        inference_initial_states = states
        
        sm_weight = tf.Variable(tf.random_uniform([num_neurons, charmap_len], minval=-0.1, maxval=0.1,name='random_uniform'),name='sm_weitht')
        sm_bias = tf.Variable(tf.random_uniform([charmap_len], minval=-0.1, maxval=0.1,name='random_uniform'),name='sm_bias')
        
        embedding = tf.Variable(tf.random_uniform([charmap_len, num_neurons], minval=-0.1, maxval=0.1,name='random_uniform'),name='embedding')

        char_input = tf.Variable(tf.random_uniform([num_neurons], minval=-0.1, maxval=0.1,name='random_uniforme'),name='char_input')
        char_input = tf.reshape(tf.tile(char_input, [n_samples]), [n_samples, 1, num_neurons],name='reshape_char_input')
        if seq_len is None:
            seq_len = tf.placeholder(tf.int32, None, name="ground_truth_sequence_length")
            
        
        if gt is not None:
            print('get_train_opt')
        else:
            inference_pred = []
            embedded_pred = [char_input]
            for i in range(seq_len):
                #step_pred, states = rnn_step_prediction(cells, charmap_len, tf.concat(embedded_pred, 1), num_neurons, seq_len,
                #                                sm_bias, sm_weight, states, reuse=reuse)
                gt_sentence_input = tf.concat(embedded_pred, 1,name='gt_sentence_input')
                with tf.variable_scope("rnn", reuse=False):
                    GRU_output = gt_sentence_input
                    for l in range(FLAGS.GEN_GRU_LAYERS):
                        GRU_output, states[l] = tf.nn.dynamic_rnn(cells[l], GRU_output, dtype=tf.float32,
                                                       initial_state=states[l], scope="layer_%d" % (l + 1))
                GRU_output = tf.reshape(GRU_output, [-1, num_neurons],name='GRU_output')
                GRU_output = tf.nn.softmax(tf.matmul(GRU_output, sm_weight) + sm_bias,name='GRU_softmax')
                GRU_output = tf.reshape(GRU_output, [BATCH_SIZE, -1, charmap_len],'reshape_GRU_output')
                step_pred = GRU_output
                #return GRU_output, states
                best_chars_tensor = tf.argmax(step_pred, axis=2,name='best_chars_tensor')
                best_chars_one_hot_tensor = tf.one_hot(best_chars_tensor, charmap_len,name='one_hot')
                best_char = best_chars_one_hot_tensor[:, -1, :]
                inference_pred.append(tf.expand_dims(best_char, 1))
                embedded_pred.append(tf.expand_dims(tf.matmul(best_char, embedding), 1,name='expand_dims'))
                reuse = True
                inference_op = tf.concat(inference_pred, axis=1,name='inference_op')
    return train_initial_states,inference_initial_states,noise   

In [218]:
tf.reset_default_graph()
train_p, infe = Generator_GRU_CL_VL_TH(64,1,1)
train_initial_states = Generator_GRU_CL_VL_TH2(64,1,1)

In [219]:
print(train_p)

None


In [220]:
print(infe)

Tensor("Generator/concat_1:0", shape=(64, 1, 1), dtype=float32)


In [221]:
with tf.Session() as sess:
  train_writer = tf.summary.FileWriter( './logs/1/train ', sess.graph) #tensorboard --logdir logs/1